# 🚀 Train Your First Custom Wake Word with Nanowakeword!

Welcome to the official tutorial for **Nanowakeword**!

In this notebook, we will guide you through the entire process of training a high-performance, custom wake word model from scratch. You don't need any pre-existing data—we will download everything we need and let Nanowakeword's intelligent engine do the heavy lifting.

**Our goal:** Go from zero to a ready-to-use wake word model in just a few simple steps. Let's get started!

<span style="color:red; font-size:26px; font-weight:bold;">
NOTE:
</span>

<span style="color:red; font-size:16px;">
This component will be updated tomorrow.  
Please do not use it until the update is completed.
</span>


**Installation**

In [1]:
# @title Step 1: Install Nanowakeword
# We install the full [train] package to get all the necessary dependencies.

# ! pip install --no-cache-dir "nanowakeword[train]==1.3.4"
! pip install "nanowakeword[train] @ git+https://github.com/arcosoph/nanowakeword.git"
! pip install piper-tts

print("Installation complete!")

  Cloning https://github.com/arcosoph/nanowakeword.git to /tmp/pip-install-bijo0e02/nanowakeword_3ff6040d985d4aee95d9a83436fb08c6
  Running command git clone --filter=blob:none --quiet https://github.com/arcosoph/nanowakeword.git /tmp/pip-install-bijo0e02/nanowakeword_3ff6040d985d4aee95d9a83436fb08c6
  Resolved https://github.com/arcosoph/nanowakeword.git to commit 50476a1dcc34f17c87d9dabfac7e63461e27bfcb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Installation complete!


## Step 2: Prepare the Dataset

A great model starts with great data. For this tutorial, we will:
1.  **Download** open-source noise and Room Impulse Response (RIR) datasets.
2.  **Organize** all your project files within a clean, well-structured folder hierarchy for better clarity and maintainability.

In [2]:
# @title Step 2: Download & Prepare the SonicWeave-v1 Dataset

import os
from pathlib import Path
import subprocess
import shutil

# --- Configuration ---
DATASET_REPO_URL = "https://huggingface.co/datasets/arcosoph/SonicWeave-v1"
DATA_DIR = Path("./nanowakeword_data")

# --- Define Final Paths ---
noise_dir = DATA_DIR / "Noise"
rir_dir = DATA_DIR / "Rir"
positive_dir = DATA_DIR / "positive_wakeword"
negative_dir = DATA_DIR / "negative_speech"

# --- Main Logic ---
print("The dataset is downloading. This may take a moment...")

# Download only if the dataset folders are not already created.
if not noise_dir.exists() or not rir_dir.exists() or not any(noise_dir.iterdir()):

    # Clone the repository to a temporary location
    temp_clone_dir = DATA_DIR / "temp_repo"

    print(f"Downloading the Starter Dataset from {DATASET_REPO_URL}...")

    # --depth 1 only downloads the latest commit, which is much faster
    try:
        subprocess.run(
            ["git", "clone", "--depth", "1", DATASET_REPO_URL, str(temp_clone_dir)],
            check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL # Hide unnecessary log messages
        )
    except subprocess.CalledProcessError as e:
        print(f"Error: Failed to clone the dataset repository. Please check the URL.")
        print(f"Git command failed with error: {e}")
    else:
        print("Organizing dataset files...")

        # Move the noise and rir folders from the cloned repository to the correct location
        try:

            # Move only required folders
            for folder_name in ["Noise", "Rir"]:
                src = temp_clone_dir / folder_name
                dst = DATA_DIR / folder_name
                if src.exists():
                    shutil.move(str(src), str(dst))

            # Delete temp_repo, ignore errors if some files are locked
            shutil.rmtree(temp_clone_dir, ignore_errors=True)

            print("\nSonicWeave-v1 Dataset is ready!")
        except FileNotFoundError:
            print("Error: 'noise' or 'rir' folder not found inside the cloned repository.")
        except Exception as e:
            print(f"Error organizing files: {e}")
else:
    print("SonicWeave-v1 Dataset already found.")

The dataset is downloading. This may take a moment...
SonicWeave-v1 Dataset already found.


## Step 3: Configure and Train the Model

Now for the fun part! We will create a `config.yaml` file and then run the Nanowakeword training command.

In [3]:
# @title Step 3.1: Create the "NanoWakeWord" Configuration File
import yaml

# Define the configuration dictionary
config_dict = {
    # ## Project & Data Paths
    # Define the project name and the locations of all data assets.
    "model_name": "arcosoph_dnn_v1",
    "output_dir": "./trained_models",

    # Source audio data paths (raw .wav files)
    "positive_data_path": str(positive_dir),
    "negative_data_path": str(negative_dir),
    "background_paths": [str(noise_dir)],
    "rir_paths": [str(rir_dir)],

    # ## Feature Manifest
    # Paths to the generated .npy feature files. The Acoustic Compositor uses these keys.
    "feature_manifest": {
        "targets": {
            "t": './trained_models/arcosoph_dnn_v1/features/positive_features_train.npy'
        },
        "negatives": {
            "n": "./trained_models/arcosoph_dnn_v1/features/negative_features_train.npy"
            # You can add unlimited additional negative sources with unique keys.
            # "my_other_negatives": "/path/to/other_negatives.npy"
        },
        "backgrounds": {
            "b": './trained_models/arcosoph_dnn_v1/features/pure_noise_features.npy'
            # For example, adding a large external noise dataset:
            # "openwakeword_noise": './openwakeword_features_ACAV100M_2000_hrs_16bit.npy'
        }
    },

    # ## Acoustic Compositor
    # Defines how to mix features from the manifest to create training scenes.
    "acoustic_compositor": {
        "use_defaults": False,
        "blueprints": [
            # 'composition' defines the scene. 'weight' defines its relative frequency.
            # You can use specific keys from the manifest (e.g., "t", "n", "openwakeword_noise", "x")
            # or general category keywords (e.g., "targets", "negatives", "backgrounds").
            # Using a category keyword like "backgrounds" will randomly select from ALL sources
            # registered under that category in the feature_manifest above.
            {"composition": ["b", "t", "b"], "weight": 40},
            {"composition": ["n", "t", "n"], "weight": 25},
            {"composition": ["n", "t", "b"], "weight": 10},
            {"composition": ["backgrounds", "t", "n"], "weight": 10},
            {"composition": ["n", "n", "n"], "weight": 30},
            {"composition": ["b", "b", "b"], "weight": 20}
            # {"composition": ["x", "my_w", "b", "best-data"], "weight": 20 } # You can add more as needed.
        ]
    },

    # ## Model Architecture
    # Configure the structure and properties of the neural network.
    "model_type": "dnn",
    "layer_size": 128,
    "n_blocks": 3,
    "embedding_dim": 64,
    "dropout_prob": 0.25,
    "activation_function": "relu",

    # ## Training & Optimizer Settings
    # Hyperparameters that control the training loop and optimization process.
    "steps": 20000,
    "batch_size": 128,
    "optimizer_type": "adamw",
    "learning_rate_max": 0.001,
    "lr_scheduler_type": "onecycle",
    "weight_decay": 0.01,
    "momentum": 0.9,

    # ## Audio Processing & Augmentation
    # Control how audio is processed and augmented to create diverse training data.
    "audio_processing": {
        "clip_length_samples": None,
        "autotune_length": {
            "enabled": True,
            "num_samples_to_inspect": 100,
            "duration_buffer_ms": 750,
            "min_allowable_length": 22000,
            "snap_to_min_tolerance": 4000
        }
    },
    "augmentation_rounds": 5,
    "augmentation_batch_size": 128,
    "min_snr_in_db": 5.0,
    "max_snr_in_db": 20.0,
    "augmentation_settings": {
        "BackgroundNoise": 0.75,
        "RIR": 0.4,
        "PitchShift": 0.3,
        "ColoredNoise": 0.25,
        "Gain": 1.0,
        "BandStopFilter": 0.1
    },

    # ## Synthetic Data Generation
    # Parameters for generating audio clips using Text-to-Speech (TTS).
    "generate_positive_samples": 2050,
    "generate_negative_samples": 4500,
    # List sensitive words or common false-alarm triggers to generate as negative data.
    "custom_negative_phrases": ['hello', 'ark', 'arcos', 'arfo', 'ar'],
    "custom_negative_per_phrase": 23,

    # ## Advanced & Miscellaneous Settings
    # Checkpointing, logging, and other fine-tuning parameters.
    "checkpoint_averaging_top_k": 5,
    "checkpointing": {
        "enabled": True,
        "interval_steps": 1000,
        "limit": 2
    },
    "early_stopping_patience": 0,
    "main_delta": 0.0001,
    "WARMUP_STEPS": 1500,
    "onnx_opset_version": 17,
    "show_training_summary": True,
    "debug_mode": True,
    "feature_gen_cpu_ratio": 1.0,
    "ema_alpha": 0.01,

    # ## Pipeline Control Switches
    # Master switches to enable or disable major stages of the pipeline.
    "generate_clips": True,
    "transform_clips": True,  # Don't forget to turn it off after finished.
    "train_model": True,
    "overwrite": False
}

# Write the config to a YAML file
config_path = "./config.yaml"
with open(config_path, 'w') as f:
    yaml.dump(config_dict, f, default_flow_style=False, sort_keys=False)

print(f"Configuration file saved to: {config_path}")


Configuration file saved to: ./config.yaml


**Run Training!**

In [ ]:
# @title Step 3.2: Run the Magic Command! 🚀
# This command will do everything: augment data, extract features, and train the model.
# It might take some time depending on the hardware (especially on a CPU).

from nanowakeword.trainer import train

args_list = [
    '--config_path', f'{config_path}',
]

print("Starting NanoWakeWord training...")

try:
    train(args_list)
    print("\n\nCONGRATULATIONS! (✿◕‿◕✿)")
    print("Your custom wake word model has been successfully trained!")

except Exception as e:
    print(f"\nAn error occurred during training: {e}")

Starting NanoWakeWord training...


  _   _               __          __   _     __          __           _ 
 | \ | |              \ \        / /  | |    \ \        / /          | |
 |  \| | __ _ _ __   __\ \  /\  / /_ _| | ____\ \  /\  / /__  _ __ __| |
 | . ` |/ _` | '_ \ / _ \ \/  \/ / _` | |/ / _ \ \/  \/ / _ \| '__/ _` |
 | |\  | (_| | | | | (_) \  /\  / (_| |   <  __/\  /\  / (_) | | | (_| |
 |_| \_|\__,_|_| |_|\___/ \/  \/ \__,_|_|\_\___| \/  \/ \___/|_|  \__,_|

STEP 1: Verifying and Preprocessing Data Directories

====================================================

INFO: 'Rir' already verified. Skipping.

...

INFO: Target negative samples: 4500. Current custom samples: 115.

INFO: Generating 4385 auto-adversarial phrases to fill the gap.

INFO: Initiating data generation pipeline for phrase: 'Hey Noah'

INFO: Executing task 'Positive_Train': 2050 clips -> 'nanowakeword_data/positive_wakeword'

Generating Audio:  79%|███████▉  | 1628/2050 [02:33<00:40, 10.43sample/s]

## What's Next?

You have successfully trained your own custom wake word model!

You can now download the `.onnx` file from the `trained_models` directory (check the file browser on the left) and use it in your own applications.

For more advanced topics, such as using your own datasets or fine-tuning the configuration, please check out our full documentation on **[GitHub](https://github.com/arcosoph/nanowakeword)**.

---
## Step 4: Save Your Model to Google Drive

The final step is to save your trained model and performance graph to a safe and accessible place. Instead of a slow direct download, we will save the files directly to your Google Drive. This process is almost instantaneous.

Run the cells below to:
1.  Connect your Google Drive account.
2.  Copy all the trained files into a new folder named `nanowakeword_models` in your Drive.

In [ ]:
# @title Step 4.1: Connect to Google Drive
# This will ask for your permission to access your Google Drive.

from google.colab import drive
import os

try:
    drive.mount('/content/drive')
    print("\nGoogle Drive connected successfully!")
except Exception as e:
    print(f"An error occurred while connecting to Google Drive: {e}")

In [ ]:
# @title Step 4.2: Copy Final Model and Artifacts to Google Drive 📂

import os
import shutil

# --- Configuration ---
# Get model_name and output_dir from the config_dict defined earlier
model_name = config_dict.get("model_name", "my_model")
output_dir = config_dict.get("output_dir", "./trained_models")

# --- Source and Destination Paths ---
# The source project directory containing all generated files
source_project_dir = os.path.join(output_dir, model_name)

# The destination folder in your Google Drive
drive_destination_dir = f"drive/MyDrive/nanowakeword_models/{model_name}"

# --- Start Copy Process ---
print("Starting the process to copy trained files to Google Drive...")

# Check if the source directory exists
if not os.path.exists(source_project_dir):
    print(f"\n❌ ERROR: Source directory not found at '{source_project_dir}'")
    print("This indicates that the training process did not create the expected output folder.")
    print("Please ensure the training step completed successfully before running this cell.")
else:
    # If an old folder exists in Drive, remove it to ensure a clean copy
    if os.path.exists(drive_destination_dir):
        print(f"🔄 Found an existing folder in Drive. Removing it for a fresh copy: '{drive_destination_dir}'")
        shutil.rmtree(drive_destination_dir)

    # --- Copy the entire project folder ---
    # This is much simpler and more reliable than copying individual files.
    # It preserves the professional directory structure.
    try:
        shutil.copytree(source_project_dir, drive_destination_dir)

        print("\n" + "="*50)
        print("✅ SUCCESS! All files have been saved to your Google Drive.")
        print("="*50)
        print(f"\nYour complete project, including the model and performance graphs, can be found in:")
        print(f"➡️ '{drive_destination_dir}'")

        # Optional: List the contents of the new folder in Drive for verification
        print("\nContents of the saved folder:")
        for root, dirs, files in os.walk(drive_destination_dir):
            level = root.replace(drive_destination_dir, '').count(os.sep)
            indent = ' ' * 4 * (level)
            print(f"{indent}{os.path.basename(root)}/")
            sub_indent = ' ' * 4 * (level + 1)
            for f in files:
                print(f"{sub_indent}{f}")

    except Exception as e:
        print(f"\n❌ ERROR: An unexpected error occurred during the copy process.")
        print(f"Details: {e}")